#### 导入库

In [1]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import os

#### 文档加载&切分&检索

In [2]:
! python ingest.py

python: can't open file '/home/linjw/AGI/llama2/langchain/ingest.py': [Errno 2] No such file or directory


In [3]:
import os
from dotenv import load_dotenv
from chromadb.config import Settings

load_dotenv()

# Define the folder for storing database
PERSIST_DIRECTORY = "/home/linjw/AGI/llama2/langchain/"
# Define the Chroma settings
CHROMA_SETTINGS = Settings(
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)

embeddings_model_name = "/home/linjw/AGI/Weight/paraphrase-multilingual-MiniLM-L12-v2"
persist_directory = "/home/linjw/AGI/privateGPT/db"
model_path = os.environ.get('MODEL_PATH')
model_n_ctx = os.environ.get('MODEL_N_CTX')
model_n_batch = int(os.environ.get('MODEL_N_BATCH',8))
target_source_chunks = int(os.environ.get('TARGET_SOURCE_CHUNKS',4))
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
chroma_client = chromadb.PersistentClient(settings=CHROMA_SETTINGS , path=persist_directory)
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS, client=chroma_client)

##### 普通检索

In [4]:
retriever = db.as_retriever(search_kwargs={"k": 100})

# retriever = db.as_retriever(    
#     search_type="mmr",
#     search_kwargs={'k': 20, 'lambda_mult': 0.25})


    参数：

    search_type（可选）：定义检索器应该执行的搜索类型。可以是 "similarity"（默认）、"mmr" 或 "similarity_score_threshold"。
    search_kwargs（可选）：传递给搜索函数的关键字参数，可以包括诸如以下内容：
    k：要返回的文档数量（默认值为 4）。
    score_threshold：用于 "similarity_score_threshold" 的最小相关性阈值。
    fetch_k：传递给 MMR 算法考虑的文档数量（默认值为 20）。
    lambda_mult：MMR 返回的结果的多样性；1 表示最小多样性，0 表示最大多样性（默认值为 0.5）。
    filter：根据文档元数据进行筛选。

In [5]:
query = "总结一下人工智能(AI)的产业规模,通过资产总量、固定资产规模、行业企业单位数量、行业资产总量增速来看产业规模。"
docs = retriever.get_relevant_documents(query)
[doc.page_content for doc in docs]

['础设施+AI 框架&算法库+应用场景”的稳定产业价值链结构。近年来众多 AI 相\n关企业亦先后登陆资本市场，亦推动二级市场 AI 产业集群效应不断凸显，基于\n产业空间、成长性、竞争格局等维度综合分析，我们判断，AI 产业将大概率成\n为全球科技领域中期最具投资价值产业赛道之一，领先 AI 芯片厂商、云计算巨\n头（算力设施+算法框架）、AI+应用场景龙头厂商、平台型算法框架厂商等有\n望持续成为产业发展核心受益者。 \n▍ 报告缘起：人工智能（AI）指利用技术学习人、模拟人，乃至超越人类智能的综\n合学科。伴随全球 AI 产业主要配套环节（芯片、软件框架、应用算法、数据积\n累）的逐步清晰、完善，以及部分简单应用场景（图像识别、语音合成、多轮会\n话、推荐系统等）的持续落地等，我们判断全球 AI 产业有望在经历前期徘徊之\n后，进入快速发展期，但作为一个极为复杂的学科和产业板块，一系列的技术、\n产业问题仍困扰着投资者。在本篇专题报告中，我们将尝试从理论技术、产业应\n用等维度，就当下 AI 产业发展的核心问题展开分析讨论，并在此基础上构建我\n们对全球 AI 产业的中期投资框架。',
 '人工智能产业人才发展报告（2019-2020 年版） \n21 \n2.1.3 企业产业链分布情况 \n在本次调研中，将企业分为基础层、技术层和应用层，其中应用层企业是此次调研中的\n主要企业群体，占比为 83.4%。在本报告中，应用层企业指的是既包括专注于人工智能领域、\n对外输出人工智能终端产品或行业解决方案的人工智能企业，又包括众多的利用人工智能重\n塑和改造自身产品和业务的信息化、互联网和传统产业企业。此外，本次调研的基础层企业\n和技术层企业的占比分别为 4.8%和 11.8%。 \n \n数据来源：报告编写组调研数据库、BOSS 直聘 \n图 10 本次调研企业的产业链分布情况 \n2.2 人工智能产业人才岗位类型 \n根据各人工智能企业岗位人才需求，可归纳为高级管理岗、高端技术岗、算法研究岗、\n应用开发岗、实际技能岗、产品经理岗等类型岗位，如图 11 所示。这一岗位需求分类也契\n合人工智能从研发到应用的众多环节。管理、技术和服务等多类型人才协同，推进人工智能\n应用落地，成为数字经济背景下人工智能产业人才内涵的特色。 \n4.8%\n11.8%\n83.

In [6]:
for doc in docs:
    print(doc.metadata)


{'author': '', 'creationDate': '', 'creator': '', 'file_path': 'source_documents/2022年  【21页】全球人工智能AI行业专题研究报告：大模型驱动AI全面提速，产业黄金十年投资周.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20220625175035+08'00'", 'page': 0, 'producer': 'Microsoft® Word 2016; modified using iText® 5.4.3 ©2000-2013 1T3XT BVBA (AGPL-version); modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': 'source_documents/2022年  【21页】全球人工智能AI行业专题研究报告：大模型驱动AI全面提速，产业黄金十年投资周.pdf', 'subject': '', 'title': '中信证券-全球人工智能AI行业专题研究报告：大模型驱动AI全面提速，产业黄金十年投资周期开启-220623.pdf', 'total_pages': 65, 'trapped': ''}
{'author': '丁 兆增', 'creationDate': "D:20200622221303+08'00'", 'creator': 'Microsoft® Word 2019', 'file_path': 'source_documents/2019-2020年人工智能产业人才发展报告.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20200622221303+08'00'", 'page': 21, 'producer': 'Microsoft® Word 2019', 'source': 'source_documents/2019-2020年人工智能产业人才发展报告.pdf', 'subject': '', 'title': '', 'total_pages': 58, 'trapped': ''}
{'author': '丁 兆增', 'cre

##### LLM辅助检索

##### LLM压缩检索

In [7]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import LLMChainExtractor
# from langchain.llms import OpenAI
# from langchain.retrievers.self_query.base import SelfQueryRetriever
# from langchain.chains.query_constructor.base import AttributeInfo
 
# #定义元数据的过滤条件
# metadata_field_info = [
#     AttributeInfo(
#         name="source",
#         description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
#         type="string",
#     ),
#     AttributeInfo(
#         name="page",
#         description="The page from the lecture",
#         type="integer",
#     ),
# ]
 
# #创建SelfQueryRetriever
# document_content_description = "Lecture notes"
# llm = OpenAI(temperature=0)
# retriever = SelfQueryRetriever.from_llm(
#     llm,
#     db,
#     document_content_description,
#     metadata_field_info,
#     verbose=True
# )
# def pretty_print_docs(docs):
#     print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))
    
# # Wrap our vectorstore
# llm = OpenAI(temperature=0)
# compressor = LLMChainExtractor.from_llm(llm)
 
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor,
#     base_retriever=db.as_retriever()
# )

# question = "what did they say about matlab?"
# compressed_docs = compression_retriever.get_relevant_documents(question)
# pretty_print_docs(compressed_docs)

#### 加载模型

In [8]:
docs

[Document(page_content='础设施+AI 框架&算法库+应用场景”的稳定产业价值链结构。近年来众多 AI 相\n关企业亦先后登陆资本市场，亦推动二级市场 AI 产业集群效应不断凸显，基于\n产业空间、成长性、竞争格局等维度综合分析，我们判断，AI 产业将大概率成\n为全球科技领域中期最具投资价值产业赛道之一，领先 AI 芯片厂商、云计算巨\n头（算力设施+算法框架）、AI+应用场景龙头厂商、平台型算法框架厂商等有\n望持续成为产业发展核心受益者。 \n▍ 报告缘起：人工智能（AI）指利用技术学习人、模拟人，乃至超越人类智能的综\n合学科。伴随全球 AI 产业主要配套环节（芯片、软件框架、应用算法、数据积\n累）的逐步清晰、完善，以及部分简单应用场景（图像识别、语音合成、多轮会\n话、推荐系统等）的持续落地等，我们判断全球 AI 产业有望在经历前期徘徊之\n后，进入快速发展期，但作为一个极为复杂的学科和产业板块，一系列的技术、\n产业问题仍困扰着投资者。在本篇专题报告中，我们将尝试从理论技术、产业应\n用等维度，就当下 AI 产业发展的核心问题展开分析讨论，并在此基础上构建我\n们对全球 AI 产业的中期投资框架。', metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': 'source_documents/2022年  【21页】全球人工智能AI行业专题研究报告：大模型驱动AI全面提速，产业黄金十年投资周.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20220625175035+08'00'", 'page': 0, 'producer': 'Microsoft® Word 2016; modified using iText® 5.4.3 ©2000-2013 1T3XT BVBA (AGPL-version); modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': 'source_documents/2022年  【21页】全球人工智能AI行业专题研究报告：大模型驱动AI全面提速，产业黄金十年投资周.pdf', 'subj

In [9]:
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
model = HuggingFacePipeline.from_model_id(model_id="/home/linjw/AGI/Weight/chinese-alpaca-2-13b-16k",
        task="text-generation",
        device=0,
        pipeline_kwargs={
            "max_new_tokens": 4096, 
            "do_sample": True,
            "temperature": 0.2,
            "top_k": 40,
            "top_p": 0.9,
            "repetition_penalty": 1.1},
        model_kwargs={
            "torch_dtype": "auto",
            "low_cpu_mem_usage": True}
        )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


##### 定义目标模板

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

prompt_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "{context}\n{question} [/INST]"
)

refine_prompt_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "这是原始问题: {question}\n"
    "已有的回答: {existing_answer}\n"
    "现在还有一些文字，（如果有需要）你可以根据它们完善现有的回答。"
    "\n\n"
    "{context_str}\n"
    "\n\n"
    "请根据新的文段，进一步完善你的回答。"
    " [/INST]"
)

initial_qa_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "以下为背景知识：\n"
    "{context_str}"
    "\n"
    "请根据以上背景知识, 回答这个问题：{question}。"
    " [/INST]"
)


In [11]:
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_prompt_template,
)
initial_qa_prompt = PromptTemplate(
    input_variables=["context_str", "question"],
    template=initial_qa_template,
)
chain_type_kwargs = {"question_prompt": initial_qa_prompt, "refine_prompt": refine_prompt}
qa = RetrievalQA.from_chain_type(
    llm=model, chain_type="refine",
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    chain_type_kwargs=chain_type_kwargs)


In [12]:
db.as_retriever(search_kwargs={"k": 10}).get_relevant_documents(query)

[Document(page_content='础设施+AI 框架&算法库+应用场景”的稳定产业价值链结构。近年来众多 AI 相\n关企业亦先后登陆资本市场，亦推动二级市场 AI 产业集群效应不断凸显，基于\n产业空间、成长性、竞争格局等维度综合分析，我们判断，AI 产业将大概率成\n为全球科技领域中期最具投资价值产业赛道之一，领先 AI 芯片厂商、云计算巨\n头（算力设施+算法框架）、AI+应用场景龙头厂商、平台型算法框架厂商等有\n望持续成为产业发展核心受益者。 \n▍ 报告缘起：人工智能（AI）指利用技术学习人、模拟人，乃至超越人类智能的综\n合学科。伴随全球 AI 产业主要配套环节（芯片、软件框架、应用算法、数据积\n累）的逐步清晰、完善，以及部分简单应用场景（图像识别、语音合成、多轮会\n话、推荐系统等）的持续落地等，我们判断全球 AI 产业有望在经历前期徘徊之\n后，进入快速发展期，但作为一个极为复杂的学科和产业板块，一系列的技术、\n产业问题仍困扰着投资者。在本篇专题报告中，我们将尝试从理论技术、产业应\n用等维度，就当下 AI 产业发展的核心问题展开分析讨论，并在此基础上构建我\n们对全球 AI 产业的中期投资框架。', metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': 'source_documents/2022年  【21页】全球人工智能AI行业专题研究报告：大模型驱动AI全面提速，产业黄金十年投资周.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20220625175035+08'00'", 'page': 0, 'producer': 'Microsoft® Word 2016; modified using iText® 5.4.3 ©2000-2013 1T3XT BVBA (AGPL-version); modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': 'source_documents/2022年  【21页】全球人工智能AI行业专题研究报告：大模型驱动AI全面提速，产业黄金十年投资周.pdf', 'subj

In [13]:
query = """
根据已有的信息总结归纳,通过总结提供资产总量、固定资产规模、行业企业单位数量和行业资产总量增速补充下面具体数值：

资产总量： 人工智能行业的资产总量为多少亿元。
固定资产规模： 人工智能行业的固定资产规模为多少亿元。
行业企业单位数量：人工智能的行业中，共有多少家企业/单位。
行业资产总量增速： 人工智能行业资产总量增速为多少 %。"""

query = """
根据已有的信息总结归纳,通过总结提供资产总量、固定资产规模、行业企业单位数量和行业资产总量增速补充下面具体数值：

资产总量： 人工智能行业的资产总量为多少亿元。
固定资产规模： 人工智能行业的固定资产规模为多少亿元。
行业企业单位数量：人工智能的行业中，共有多少家企业/单位。
行业资产总量增速： 人工智能行业资产总量增速为多少 %。"""


qa.run(query)

' 根据提供的新文本，我们可以得出以下结论来补充已有的答案：\n\n- 行业企业单位数量：根据所提到的数据，从2014年至今，布局人工智能赛道的投资机构数量呈上升趋势，每年增长率约为39.4%。这表明人工智能行业吸引了越来越多的企业参与其中。\n\n- 行业资产总量增速：根据所提供的信息，人工智能行业在最近几年内一直保持快速增长的趋势。2018年，该行业的资产总量已经突破1000亿人民币，预计未来将继续保持高速增长。此外，政策的支持以及资本市场对人工智能行业的关注度都在增加，这也有助于推动行业的发展。\n\n综上所述，人工智能行业的总资产量可能接近或超过1000亿人民币，而固定资产规模则可能达到数百亿级别。至于具体的数字，由于缺乏明确的数据来源，无法给出准确的估算。同时，该行业中的企业数量也在逐渐增多，但确切的数字也需要更多的详细统计数据才能确定。'

In [14]:
qa_chain_refine = RetrievalQA.from_chain_type(
    model,
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    chain_type="refine"
)
 
result = qa_chain_refine({"query": query})
result["result"]


/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


"\n\nThe original answer is as follows:\n\nAs per the latest update, the total asset value of the artificial intelligence industry was estimated to be around 7,369.5 billion yuan (approximately $1,070 billion) in 2019. However, the specific values for the industry may have changed since then due to various factors such as technological advancements, market demand, and regulatory policies. It is recommended to conduct further research and analysis to obtain more accurate and up-to-date information about the industry's assets, scale, and growth.\n\nIn terms of specific values for the artificial intelligence industry, the total asset value of the core industry was estimated to be around 1,418.5 billion yuan (approximately $200 billion), while the total asset value of the supporting layer industry was estimated to be around 736.95 billion yuan (approximately $107 billion). The software component of the industry still had a relatively small share in the total asset value, accounting for onl

In [15]:
result

{'query': '\n根据已有的信息总结归纳,通过总结提供资产总量、固定资产规模、行业企业单位数量和行业资产总量增速补充下面具体数值：\n\n资产总量： 人工智能行业的资产总量为多少亿元。\n固定资产规模： 人工智能行业的固定资产规模为多少亿元。\n行业企业单位数量：人工智能的行业中，共有多少家企业/单位。\n行业资产总量增速： 人工智能行业资产总量增速为多少 %。',
 'result': "\n\nThe original answer is as follows:\n\nAs per the latest update, the total asset value of the artificial intelligence industry was estimated to be around 7,369.5 billion yuan (approximately $1,070 billion) in 2019. However, the specific values for the industry may have changed since then due to various factors such as technological advancements, market demand, and regulatory policies. It is recommended to conduct further research and analysis to obtain more accurate and up-to-date information about the industry's assets, scale, and growth.\n\nIn terms of specific values for the artificial intelligence industry, the total asset value of the core industry was estimated to be around 1,418.5 billion yuan (approximately $200 billion), while the total asset value of the supporting layer indu

#### 解析最终输出

#### 输出修复器

In [47]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
 

#### Pydantic

In [89]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
 

In [90]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
 
    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field
 
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
 
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)
 
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
 
_input = prompt.format_prompt(query=joke_query)
 
output = model(_input.to_string())
 
parser.parse(output)
 

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

In [95]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")
 
actor_query = "Generate the filmography for a random actor."
 
parser = PydanticOutputParser(pydantic_object=Actor)
 
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
 
_input = prompt.format_prompt(query=actor_query)
 
output = model(_input.to_string())
 
parser.parse(output)
 

Actor(name='John Smith', film_names=['The Matrix', 'The Dark Knight Rises', 'Inception', 'Interstellar', 'The Departed'])

#### 数据解析器

In [124]:
query = """
根据已有的信息总结归纳,通过总结提供资产总量、固定资产规模、行业企业单位数量和行业资产总量增速补充下面具体数值：

资产总量： 人工智能行业的资产总量为多少亿元。
固定资产规模： 人工智能行业的固定资产规模为多少亿元。
行业企业单位数量：人工智能的行业中，共有多少家企业/单位。
行业资产总量增速： 人工智能行业资产总量增速为多少 %。"""

query = """ 将背景知识里的人工智能产业的资产总量、固定资产规模、行业企业单位数量、行业资产总量增速的值提取出来(允许估算,如果提取不出来则返回0),并按照模板进行返回"""



In [128]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

que = """我们可以得出以下结论来补充已有的答案：\n\n- 行业企业单位数量：根据所提到的数据，从2014年至今，布局人工智能赛道的投资机构数量呈上升趋势，
每年增长率约为39.4%。这表明人工智能行业吸引了越来越多的企业参与其中。\n\n- 行业资产总量增速：根据所提供的信息，人工智能行业在最近几年内一直保持快速增长的趋势。
2018年，该行业的资产总量已经突破1000亿人民币，预计未来将继续保持高速增长。此外，政策的支持以及资本市场对人工智能行业的关注度都在增加，这也有助于推动行业的发展。\n\n综上所述，人工智能行业的总资产量可能接近或超过1000亿人民币，
而固定资产规模则可能达到数百亿级别。至于具体的数字，由于缺乏明确的数据来源，无法给出准确的估算。同时，该行业中的企业数量大概在30000个左右，但确切的数字也需要更多的详细统计数据才能确定"""

initial_qa_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "以下为背景知识：\n"
    "{context_str}"
    "\n"
    "{format_instructions}"
    "\n"
    "请根据以上背景知识, 回答这个问题：{question}。"
    " [/INST]"
)

response_schemas = [
    ResponseSchema(name="asset_total", description="人工智能行业的大概资产总量(亿元)"),
    ResponseSchema(name="fixed_asset_scale", description="人工智能行业的大概固定资产规模(亿元)"),
    ResponseSchema(name="enterprise_unit_number", description="人工智能的行业中，大概多少家企业/单位(个)"),
    ResponseSchema(name="growth_rate", description="人工智能行业的每年增长率(百分比)"),
    
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=initial_qa_template,
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions,"context_str":que}
)
_input = prompt.format_prompt(question=query)
output = model(_input.to_string())
print(output)
output_parser.parse(output)
 

 ```json
{
    "asset_total": 1000,
    "fixed_asset_scale": 500,
    "enterprise_unit_number": 10000,
    "growth_rate": 39.4
}
```


{'asset_total': 1000,
 'fixed_asset_scale': 500,
 'enterprise_unit_number': 10000,
 'growth_rate': 39.4}